In [198]:
import keras
from keras.models import Sequential, Model
import keras.backend as K
from keras.layers import Dense, Dropout, Activation,Input, Merge, add
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import random
import tensorflow as tf

In [199]:
data = pd.read_csv('../Data/trainset_impute50000.csv')
x = data.drop(data.columns[[0, 1]], axis=1)
x = np.array(x)
y = data['target']
y = np.array(y)
random.seed(867)
train_index = np.random.choice(range(data.shape[0]),int(0.8*data.shape[0]))
validate_index=set(range(data.shape[0]))-set(train_index)
validate_index=np.array(list(validate_index),dtype='int64')
#
x_train = x[train_index,:]
x_validate = x[validate_index,:]
y_train = y[train_index]
y_validate = y[validate_index]
#
case_index = [i for i,t in enumerate(y_train) if t == 1]
control_index = [i for i,t in enumerate(y_train) if t == 0]
#
new_control_index = np.random.choice(control_index,4*len(case_index))
new_index = np.concatenate((new_control_index,case_index))
new_index
#
new_x_train = x_train[new_index,:]
new_y_train = y_train[new_index]
# Convert to one-hot-vector
nb_classes = 2
#OH_new_y_train = np.eye(nb_classes)[y_train]
OH_new_y_train = np.eye(nb_classes)[new_y_train]
OH_y_validate = np.eye(nb_classes)[y_validate]

In [200]:
print(sum(y_train==0)/sum(y_train==1))

26.6344379316


In [201]:
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
# Fit only to the training data
scaler.fit(new_x_train)

new_x_train = scaler.transform(new_x_train)
#new_x_train = scaler.transform(x_train)
x_validate = scaler.transform(x_validate)

In [202]:
def Gini(OH_y_trues, OH_y_preds):
    sess = tf.Session()
    with sess.as_default():
        y_true = tf.constant(OH_y_trues).eval(shape=(2, ), dtype=float32)
        y_pred = tf.Variable(OH_y_preds).eval(shape=(2, ), dtype=float32)
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    
    # sort rows on prediction column 
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:,0].argsort()][::-1,0]
    pred_order = arr[arr[:,1].argsort()][::-1,0]
    
    # get Lorenz curves
    L_true = np.cumsum(true_order) / np.sum(true_order)
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)
    L_ones = np.linspace(1/n_samples, 1, n_samples)
    
    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)
    
    # normalize to true Gini coefficient
    val = G_pred/G_true
    var = K.variable(value=val)
    return(var)



In [203]:
def mean_pred(y_true, y_pred):
    m = np.mean(y_true)
    var = K.variable(value=m)
    return (var)
mean_pred(np.array([0,1]),np.array([0,1]))

<tf.Variable 'Variable_1:0' shape=() dtype=float32_ref>

In [164]:
sess = tf.Session()
print(type(tf.constant(OH_y_true)))
with sess.as_default():
    OH_y_true = tf.constant(OH_y_true).eval()
#    OH_y_pred = OH_y_pred.eval()
    print(type(OH_y_true))

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.ndarray'>


In [204]:
import keras.backend as K
import tensorflow as tf
def gini(actual, pred):
    pred = tf.argmax(pred, axis=1)
    actual = tf.argmax(actual, axis=1)
    nT = K.shape(pred)[0]
    n = K.cast(nT, dtype='int32')
    n_float = K.cast(nT, dtype=K.floatx())
    actual = K.reshape(actual,(1,n))[-1]
    pred = K.reshape(pred,(1,n))[-1]
    inds = tf.nn.top_k(pred, n)[1]
    a_s = K.gather(actual, inds)
    a_c = K.cumsum(a_s)
    s1 = K.sum(a_c)
    s2 = K.sum(a_s)
    giniSum = K.cast(tf.divide(s1,s2),dtype=K.floatx()) - K.cast(tf.divide(n+1,2),dtype=K.floatx())
    standard_gini = K.cast(tf.divide(giniSum,n_float),dtype=K.floatx())
    return standard_gini

def gini_normalized(a, p):
    norm_gini = K.cast(tf.divide(gini(a, p),gini(a, a)),dtype=K.floatx())
    return norm_gini

## new loss
## pred  actual  loss
##  0       1      9
##  1       0      1 
##  1/0     1/0    0
##  a=5,b=-4,c=0

def new_loss(actual,pred):
    return K.mean(5*K.square(pred-actual)-4*(pred-actual),axis=-1)

In [205]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced', np.unique(new_y_train), new_y_train)
class_weight

array([ 0.625,  2.5  ])

In [207]:

model = Sequential()
model.add(Dense(400, input_dim=np.shape(x)[1], activation='relu'))
model.add(Dense(200, input_dim=np.shape(x)[1], activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy',gini_normalized])

model.fit(new_x_train, OH_new_y_train,epochs=20,batch_size=200 ,#,class_weight = {0:0.51901583 , 1:13.64693913},
          validation_data=(x_validate, OH_y_validate))
model.evaluate(x_validate, OH_y_validate, batch_size=1000 )

Train on 86155 samples, validate on 267355 samples
Epoch 1/20
86155/86155 [==============================] - 10s 110us/step - loss: 0.4890 - acc: 0.7996 - gini_normalized: 0.0036 - val_loss: 0.2554 - val_acc: 0.9618 - val_gini_normalized: -0.0029
Epoch 2/20
86155/86155 [==============================] - 8s 92us/step - loss: 0.4715 - acc: 0.8015 - gini_normalized: 0.0217 - val_loss: 0.3015 - val_acc: 0.9495 - val_gini_normalized: 0.0169
Epoch 3/20
86155/86155 [==============================] - 8s 96us/step - loss: 0.4542 - acc: 0.8058 - gini_normalized: 0.0669 - val_loss: 0.2925 - val_acc: 0.9452 - val_gini_normalized: 0.0214
Epoch 4/20
86155/86155 [==============================] - 9s 99us/step - loss: 0.4323 - acc: 0.8144 - gini_normalized: 0.1180 - val_loss: 0.2763 - val_acc: 0.9413 - val_gini_normalized: 0.0180
Epoch 5/20
86155/86155 [==============================] - 9s 100us/step - loss: 0.4080 - acc: 0.8249 - gini_normalized: 0.1865 - val_loss: 0.2472 - val_acc: 0.9434 - val_gini

[0.41044083386015878, 0.87821061852153492, 0.044885182301120825]

In [58]:
# Fit the model on the batches generated by datagen.flow().
model1.fit_generator(datagen.flow(new_x_train, OH_new_y_train,
                    batch_size=1000),
                    samples_per_epoch=new_x_train.shape[0],
                    nb_epoch=40,
                    validation_data=(x_validate, OH_y_validate))

NameError: name 'datagen' is not defined

In [197]:

##
model = Sequential()
model.add(Dense(400, input_dim=np.shape(x)[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(200, input_dim=np.shape(x)[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, input_dim=np.shape(x)[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-2, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy',gini_normalized])

model.fit(new_x_train, OH_new_y_train,epochs=10,batch_size=200)

Tensor("metrics_61/gini_normalized/Shape:0", shape=(2,), dtype=int32)
Tensor("metrics_61/gini_normalized/Shape_2:0", shape=(2,), dtype=int32)
Epoch 1/10
86150/86150 [==============================] - 6s 75us/step - loss: 0.5189 - acc: 0.7965 - gini_normalized: 0.6007
Epoch 2/10
86150/86150 [==============================] - 5s 58us/step - loss: 0.5036 - acc: 0.7999 - gini_normalized: 0.5868
Epoch 3/10
86150/86150 [==============================] - 5s 55us/step - loss: 0.5007 - acc: 0.7998 - gini_normalized: 0.5369
Epoch 4/10
86150/86150 [==============================] - 5s 56us/step - loss: 0.5005 - acc: 0.7999 - gini_normalized: 0.5926
Epoch 5/10
86150/86150 [==============================] - 5s 59us/step - loss: 0.4994 - acc: 0.7999 - gini_normalized: 0.6019
Epoch 6/10
86150/86150 [==============================] - 5s 58us/step - loss: 0.4988 - acc: 0.8000 - gini_normalized: 0.5961
Epoch 7/10
86150/86150 [==============================] - 5s 58us/step - loss: 0.4987 - acc: 0.7999 - 

KeyboardInterrupt: 